In [2]:
import HFH

DEVICE: GPU


u:\Files\LamaIndex\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-small-en
Load pretrained SentenceTransformer: BAAI/bge-small-en
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


##### Data Source
- 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf'
- 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf'

In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader


def create_index(file_path):
    documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
    index = VectorStoreIndex.from_documents(documents)
    return index


lyft_index = create_index("./data/10k/lyft_2021.pdf")
uber_index = create_index("./data/10k/uber_2021.pdf")

Batches: 100%|██████████| 1/1 [00:00<00:00,  3.02it/s]


In [4]:
from llama_index.core.agent import ReActAgent,AgentRunner
from llama_index.core.tools import QueryEngineTool, ToolMetadata

# NOTE: lyft_index and uber_index are both SimpleVectorIndex instances
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description="Provides information about Lyft financials for year 2021. "
            "Use a detailed plain text question as input to the tool.",
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description="Provides information about Uber financials for year 2021. "
            "Use a detailed plain text question as input to the tool.",
        ),
    ),
]

# initialize agent
# agent = ReActAgent.from_tools(query_engine_tools, verbose=True)
agent = AgentRunner.from_llm(query_engine_tools, verbose=True)

In [6]:
response = agent.query("what is lyft ?")
from IPython.display import Markdown, display

display(Markdown(f"{response}"))

Thought: The current language of the user is: en. I need to use a tool to help me answer the question.
Action: lyft_10k
Action Input: {'input': 'What is lyft ?'}


Batches: 100%|██████████| 1/1 [00:00<00:00, 22.09it/s]


Observation: Lyft is a company that operates multimodal transportation networks in the United States and Canada that offer access to various transportation options through the company's platform and mobile-based applications.
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Lyft is a company that operates multimodal transportation networks in the United States and Canada that offer access to various transportation options through the company's platform and mobile-based applications.


Lyft is a company that operates multimodal transportation networks in the United States and Canada that offer access to various transportation options through the company's platform and mobile-based applications.

In [7]:
response = agent.query("what is uber ?")
from IPython.display import Markdown, display

display(Markdown(f"{response}"))

Thought: The current language of the user is: english. I need to use a tool to help me answer the question.
Action: uber_10k
Action Input: {'input': 'What is uber?'}


Batches: 100%|██████████| 1/1 [00:00<00:00, 27.52it/s]


Observation: Uber is a technology platform that uses a massive network, leading technology, operational excellence, and product expertise to power movement from point A to point B.
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Uber is a technology platform that uses a massive network, leading technology, operational excellence, and product expertise to power movement from point A to point B.


Uber is a technology platform that uses a massive network, leading technology, operational excellence, and product expertise to power movement from point A to point B.